Let's try to run a tabular model with just pytorch -- having alot of issues injecting quantile regression into a fastai for some reason

In [9]:
from fastai.tabular.all import *
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
#next two lines needed to use pytorch object vs fastai
from torch.utils.data import Dataset as torch_ds 
from torch.utils.data import DataLoader as torch_dl
from torch.utils.data import Subset as torch_subset
%matplotlib inline

In [10]:
path = Path('/home/azaidi/Desktop/fastai/nbs/kaggle/osic')
Path.BASE_PATH = path
path.ls()

(#6) [Path('test.csv'),Path('train'),Path('train.csv'),Path('test'),Path('osic-pulmonary-fibrosis-progression.zip'),Path('sample_submission.csv')]

In [11]:
train_df = pd.read_csv(path/'train.csv')
sample_sub = pd.read_csv(path/'sample_submission.csv')
test_df = pd.read_csv(path/'test.csv')

In [12]:
splits = RandomSplitter(valid_pct=0.1)(range_of(train_df))
splits

((#1395) [336,509,626,160,405,1045,655,699,1358,1202...],
 (#154) [924,274,712,1261,1031,958,557,640,267,962...])

In [13]:
test_df.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
0,ID00419637202311204720264,6,3020,70.186855,73,Male,Ex-smoker
1,ID00421637202311550012437,15,2739,82.045291,68,Male,Ex-smoker
2,ID00422637202311677017371,6,1930,76.672493,73,Male,Ex-smoker
3,ID00423637202312137826377,17,3294,79.258903,72,Male,Ex-smoker
4,ID00426637202313170790466,0,2925,71.824968,73,Male,Never smoked


In [14]:
train_df

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker
...,...,...,...,...,...,...,...
1544,ID00426637202313170790466,13,2712,66.594637,73,Male,Never smoked
1545,ID00426637202313170790466,19,2978,73.126412,73,Male,Never smoked
1546,ID00426637202313170790466,31,2908,71.407524,73,Male,Never smoked
1547,ID00426637202313170790466,43,2975,73.052745,73,Male,Never smoked


In [15]:
to = TabularPandas(train_df, procs=[Categorify, Normalize],
                  cat_names = ['Patient','Sex', 'SmokingStatus'],
                  cont_names = ['Weeks', 'Age', 'Percent'],
                  y_names = 'FVC',
                  splits=splits)

In [16]:
dls = to.dataloaders(bs=12)

In [17]:
dls.show_batch()

,Patient,Sex,SmokingStatus,Weeks,Age,Percent,FVC
0,ID00104637202208063407045,Male,Never smoked,20.000000,74.0,68.493545,2377.0
1,ID00192637202245493238298,Female,Never smoked,14.000000,56.0,61.034394,1874.0
2,ID00275637202271440119890,Female,Never smoked,52.000000,62.0,84.787537,2133.0
3,ID00061637202188184085559,Male,Ex-smoker,26.000000,68.0,113.012573,4134.0
4,ID00264637202270643353440,Female,Never smoked,39.000000,76.0,120.918127,2044.0
5,ID00099637202206203080121,Male,Ex-smoker,44.000000,68.0,82.212509,2934.0
6,ID00233637202260580149633,Male,Ex-smoker,6.000001,68.0,94.072433,3574.0
7,ID00023637202179104603099,Female,Ex-smoker,27.000000,71.0,56.845237,1337.0
8,ID00109637202210454292264,Male,Ex-smoker,107.000000,73.0,44.898491,1725.0
9,ID00367637202296290303449,Female,Never smoked,59.000000,57.0,54.240471,1329.0


In [18]:
learn = tabular_learner(dls, metrics=mse)
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,mse,time
0,8039868.000000,8171016.500000,8171016.500000,00:00
1,7619932.000000,7346683.000000,7346683.000000,00:00
2,4789520.000000,3197957.750000,3197957.750000,00:00
3,2011527.125000,1099065.875000,1099065.875000,00:00
4,969001.062500,569952.437500,569952.437500,00:00
5,663963.875000,386250.687500,386250.687500,00:00
6,642316.937500,226774.312500,226774.312500,00:00
7,564938.812500,204515.140625,204515.140625,00:00
8,610736.625000,227625.171875,227625.171875,00:00
9,616494.625000,214870.765625,214870.765625,00:00


In [47]:
learn.loss_func

FlattenedLoss of MSELoss()

In [19]:
learn.show_results()

,Patient,Sex,SmokingStatus,Weeks,Age,Percent,FVC,FVC_pred
0,138.0,2.0,2.0,-1.123116,-1.592018,-0.625601,2569.0,2453.156494
1,102.0,2.0,2.0,-0.778416,0.105134,0.995006,3685.0,3270.119873
2,148.0,2.0,2.0,-1.209291,-0.319154,-1.500948,2030.0,1788.352417
3,154.0,2.0,2.0,-0.304454,1.095140,-0.837852,2209.0,1919.354858
4,93.0,2.0,2.0,1.160520,1.943716,0.673444,2802.0,2452.015381
5,91.0,2.0,2.0,-1.036941,0.246564,0.406206,2929.0,2479.094238
6,7.0,2.0,2.0,-0.045929,0.529422,-0.038744,2968.0,2712.130371
7,96.0,2.0,2.0,-1.080028,0.387993,-0.932176,2492.0,2482.152100
8,20.0,1.0,3.0,0.040246,0.387993,-0.802466,1414.0,535.798340


In [20]:
learn.fit_one_cycle(40, lr=1e-1)

epoch,train_loss,valid_loss,mse,time
0,631569.187500,167520.296875,167520.296875,00:00
1,771233.812500,264516.531250,264516.531250,00:00
2,572872.750000,175710.109375,175710.109375,00:00
3,802426.750000,221709.109375,221709.109375,00:00
4,699228.625000,259641.109375,259641.109375,00:00
5,604095.812500,229898.812500,229898.812500,00:00
6,535797.187500,217561.656250,217561.656250,00:00
7,598414.750000,517768.531250,517768.531250,00:00
8,476458.937500,161881.312500,161881.312500,00:00
9,475002.406250,154937.578125,154937.578125,00:00


In [21]:
learn.show_results()

,Patient,Sex,SmokingStatus,Weeks,Age,Percent,FVC,FVC_pred
0,109.0,2.0,2.0,0.341858,-1.592018,0.730997,3246.0,2933.674805
1,49.0,2.0,1.0,1.203607,0.105134,0.252342,3037.0,2698.418457
2,173.0,2.0,2.0,-0.390629,0.105134,0.406549,2853.0,2635.818359
3,114.0,2.0,2.0,1.203607,-1.026300,0.047683,2962.0,2609.991699
4,25.0,2.0,3.0,0.686558,0.953710,0.014941,2341.0,2330.601807
5,67.0,2.0,2.0,-0.218279,1.236569,1.536025,4050.0,3625.449951
6,70.0,2.0,2.0,-1.036941,-0.602012,-0.468956,2841.0,2717.761230
7,80.0,2.0,2.0,0.471121,-2.157735,-0.469990,3175.0,2922.274658
8,49.0,2.0,1.0,-0.864591,0.105134,-0.031561,2832.0,2722.284180


The predsictions actually are getting better, nice. But we need to be providing a confidence value(!!!!) as well as just predicting the FVC. The confidence value is basically the std deviation. Looks like alot of people are trying to implement this with a quantile loss function. 

Let's now implement a loss function better than MSE and see the actual metric of the competition.

Laplace Log Likelihood:

σclipped=max(σ,70)<br>
Δ=min(|FVCtrue−FVCpredicted|,1000)<br>
metric=−2‾√Δσclipped−ln(2‾√σclipped).

In [23]:
std = np.std(train_df.FVC)
std

832.5021066817243

In [24]:
actual_fvc = train_df.FVC
pred_fvc = np.mean(train_df.FVC)
pred_fvc

2690.479018721756

In [25]:
sd_clipped = np.maximum(np.std(train_df.FVC), 70)
sd_clipped

832.5021066817243

In [26]:
delta = np.abs(actual_fvc - pred_fvc) * np.sqrt(2)
delta

0       531.007521
1       673.843090
2       890.217766
3       772.838040
4       878.904057
           ...    
1544     30.435264
1545    406.616071
1546    307.621122
1547    402.373431
1548    118.116504
Name: FVC, Length: 1549, dtype: float64

In [176]:
laplace_log = np.sqrt(2) * delta / sd_clipped - np.log(np.sqrt(2) * sd_clipped)
laplace_log.sum()

-8514.043756623654

Now let's look at the quantile loss fxn we'll use

In [28]:
quantiles = (0.2, 0.5, 0.8)

In [32]:
#some hypothetical prediction of one sample for the 3 quantiles
quant_pred_fvc = (pred_fvc*0.82, pred_fvc, pred_fvc*1.23)
quant_pred_fvc

(2206.19279535184, 2690.479018721756, 3309.2891930277597)

In [34]:
for i, q in enumerate(quantiles):
    print("i is: ", i)
    print("q is: ", q)

i is:  0
q is:  0.2
i is:  1
q is:  0.5
i is:  2
q is:  0.8


In [37]:
target = actual_fvc[0]
target

2315

In [39]:
target - quant_pred_fvc

array([ 108.80720465, -375.47901872, -994.28919303])

In [40]:
error = target - quant_pred_fvc[0]
error

108.80720464816022

In [44]:
q, (q - 1) * error, q*error

(0.8, -21.76144092963204, 87.04576371852818)

In [45]:
max((q-1)* error, q*error)

87.04576371852818

In [140]:
def quantile_loss(preds, target, quantiles=(0.2, 0.5, 0.8)):
    losses = []
    for i, q in enumerate(quantiles):
        errors = target - preds[:, i]
        losses.append(torch.max((q - 1) * errors, q * errors).unsqueeze(1))
    loss = torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))
    return loss

In [148]:
??tabular_learner

In [141]:
learn = tabular_learner(dls, loss_func=quantile_loss, n_out=3)

In [142]:
learn.summary()

TabularModel (Input shape: ['12 x 3', '12 x 3'])
Layer (type)         Output Shape         Param #    Trainable 
Embedding            12 x 29              5,133      True      
________________________________________________________________
Embedding            12 x 3               9          True      
________________________________________________________________
Embedding            12 x 3               12         True      
________________________________________________________________
Dropout              12 x 35              0          False     
________________________________________________________________
BatchNorm1d          12 x 3               6          True      
________________________________________________________________
BatchNorm1d          12 x 38              76         True      
________________________________________________________________
Linear               12 x 200             7,600      True      
_________________________________________________

In [143]:
learn.fit_one_cycle(200)

epoch,train_loss,valid_loss,time
0,4052.146729,4107.108887,00:00
1,4002.810547,4106.704590,00:00
2,4010.572510,4106.118652,00:00
3,4003.869629,4105.313965,00:00
4,4006.013916,4103.941895,00:00
5,4027.311768,4101.002441,00:00
6,4001.252686,4094.334473,00:00
7,3981.444824,4080.745117,00:00
8,3988.569824,4054.661865,00:00
9,3938.560059,4013.477295,00:00


In [118]:
one_batch = dls.one_batch()

In [144]:
learn.dls.test_dl(test_df)

In [145]:
test_dl = learn.dls.test_dl(test_df)

In [146]:
cat, cont, y = next(iter(test_dl))

In [147]:
learn.model(cat, cont)

tensor([[1978.3484, 2583.1653, 3253.5005],
        [2013.1833, 2614.4062, 3284.8748],
        [1967.7228, 2559.2102, 3218.0344],
        [1973.1368, 2570.3574, 3234.3755],
        [1977.9271, 2581.0354, 3250.0500]], device='cuda:0',
       grad_fn=<AddmmBackward>)

In [139]:
y

tensor([[3020],
        [2739],
        [1930],
        [3294],
        [2925]], device='cuda:0')

In [151]:
learn = tabular_learner(dls, lr=3e-3,
                        loss_func=quantile_loss, n_out=3)

In [152]:
learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,time
0,3951.208252,3547.120850,00:00
1,1636.151245,919.371765,00:00
2,1154.435425,908.367188,00:00
3,1051.687866,875.845093,00:00
4,976.339050,870.554688,00:00


In [153]:
learn.fit_one_cycle(5, lr=3e-4)

epoch,train_loss,valid_loss,time
0,950.093750,838.371399,00:00
1,844.172791,822.316223,00:00
2,826.215820,812.743530,00:00
3,833.161377,811.481445,00:00
4,811.030090,807.440979,00:00


In [154]:
learn.fit_one_cycle(10, lr=3e-5)

epoch,train_loss,valid_loss,time
0,821.694946,811.227295,00:00
1,812.806946,811.744141,00:00
2,812.332886,829.354797,00:00
3,833.972046,812.126587,00:00
4,799.394531,816.422180,00:00
5,803.059082,809.490906,00:00
6,794.651184,802.226501,00:00
7,818.753723,799.324524,00:00
8,809.737488,812.579651,00:00
9,789.333923,806.992798,00:00


In [155]:
test_dl = learn.dls.test_dl(test_df)
cat, cont, y = next(iter(test_dl))
learn.model(cat, cont)

tensor([[1893.5212, 2528.6118, 3181.5300],
        [1841.6361, 2460.3394, 3107.4668],
        [1939.6361, 2604.7273, 3343.2151],
        [1858.4597, 2482.8713, 3140.6108],
        [1881.3634, 2509.9016, 3142.4504]], device='cuda:0',
       grad_fn=<AddmmBackward>)

In [156]:
y

tensor([[3020],
        [2739],
        [1930],
        [3294],
        [2925]], device='cuda:0')

In [180]:
#@staticmethod
def laplace(preds, targets):
        sigma = preds[:, 2] - preds[:, 0]
        sigma[sigma < 70] = 70
        delta = (preds[:, 1] - targets).abs()
        delta[delta > 1000] = 1000
        return (-np.sqrt(2) * delta / sigma - torch.log(np.sqrt(2) * sigma)).mean()

In [196]:
learn = tabular_learner(dls, loss_func=quantile_loss, 
                        metrics=laplace, n_out=3)

In [197]:
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,laplace,time
0,4032.513184,4105.645996,-24.776615,00:00
1,3984.566162,3937.504395,-24.759245,00:00
2,3062.520752,2433.492188,-22.834597,00:00
3,1658.501099,1063.911621,-8.405238,00:00
4,1243.051636,876.306519,-8.081571,00:00
5,1093.044678,851.723511,-8.141219,00:00
6,1011.046021,835.683289,-8.159774,00:00
7,1085.663940,823.003174,-8.174395,00:00
8,1112.089600,818.751953,-8.181651,00:00
9,1106.873535,827.532654,-8.187715,00:00


In [192]:
def laplace_log(preds, targets):
    std = preds[:, 2] - preds[:, 0]
    std[std < 70] = 70
    delta = (preds[:, 1] - targets).abs()
    delta[delta > 1000] = 1000
    laplace = -np.sqrt(2) * delta / std - torch.log(np.sqrt(2) * std)
    return laplace.mean()

In [198]:
learn = tabular_learner(dls, loss_func=quantile_loss, 
                        metrics=laplace, n_out=3)
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,laplace,time
0,4043.902588,4105.826660,-24.776602,00:00
1,3974.207764,3902.052246,-24.746752,00:00
2,2973.898682,2310.448486,-22.372540,00:00
3,1709.089355,1108.738892,-8.361071,00:00
4,1230.134766,889.778381,-8.088428,00:00
5,1096.862183,834.369751,-8.134984,00:00
6,1093.472656,842.052612,-8.169194,00:00
7,1141.002930,816.469421,-8.198761,00:00
8,1141.389771,817.753418,-8.195677,00:00
9,1138.317871,819.122559,-8.198830,00:00


In [199]:
to_2 = TabularPandas(train_df, procs=[Categorify, Normalize],
                  cat_names = ['Sex', 'SmokingStatus'],
                  cont_names = ['Weeks', 'Age', 'Percent'],
                  y_names = 'FVC',
                  splits=splits)
dls2 = to_2.dataloaders(bs=32)
dls2.show_batch()

,Sex,SmokingStatus,Weeks,Age,Percent,FVC
0,Male,Ex-smoker,14.000000,73.0,72.748566,2795.0
1,Male,Ex-smoker,33.000000,67.0,62.197384,2415.0
2,Male,Never smoked,5.000001,51.0,50.087226,2182.0
3,Male,Ex-smoker,55.000000,71.0,72.261070,2356.0
4,Male,Ex-smoker,59.000000,73.0,63.377334,2727.0
5,Female,Never smoked,18.999999,77.0,50.493797,997.0
6,Female,Never smoked,38.000000,57.0,59.872662,1467.0
7,Male,Ex-smoker,9.000000,65.0,89.921501,3849.0
8,Male,Ex-smoker,17.000000,71.0,139.741747,5844.0
9,Male,Ex-smoker,37.000000,60.0,66.519684,2474.0


In [201]:
learn = tabular_learner(dls2, loss_func=quantile_loss, 
                        metrics=laplace, n_out=3)
learn.fit_one_cycle(100)

epoch,train_loss,valid_loss,laplace,time
0,4033.067139,4107.351562,-24.776714,00:00
1,4024.670410,4107.034668,-24.776684,00:00
2,4024.379639,4106.400391,-24.776627,00:00
3,4020.659424,4105.666504,-24.776566,00:00
4,4022.622559,4104.299316,-24.776457,00:00
5,4019.715576,4101.657227,-24.776234,00:00
6,4023.807617,4096.457031,-24.775776,00:00
7,4012.818604,4085.992188,-24.774797,00:00
8,3999.943359,4065.709473,-24.772961,00:00
9,3990.567627,4027.963379,-24.768692,00:00
